[![Open Rendered Output](https://img.shields.io/badge/Rendered%20Output-Open-blue?logo=link&logoColor=white)](https://htmlpreview.github.io/?https://raw.githubusercontent.com/MeteoSwiss/nwp-fdb-polytope-demo/main/notebooks/snapshots/full_field.html)

# ICON-CH1-EPS Full Horizontal Field with Polytope & earthkit

The following notebook demonstrates the full workflow to access, process and visualize ICON-CH1-EPS model data on the full horizontal field using 2-meter temperature data. 

<div style="text-align:center;">
  <img src="https://raw.githubusercontent.com/MeteoSwiss/nwp-fdb-polytope-demo/main/notebooks/Polytope/images/t_2m_full_field.png" style="width:70%;"/>
</div>

The data is retrieved using [Polytope](https://polytope.readthedocs.io/en/latest/), a feature extraction software developed by ECMWF. It applies concepts of computational geometry to extract n-dimensional polygons (also known as polytopes) from datacubes, also including the full horizontal field. To access MeteoSwiss' operational ICON-CH1-EPS and ICON-CH2-EPS model data, [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) provides a wrapper around the Polytope client that simplifies the request API. Follow the instructions to learn more about model data access via Polytope.

## Installation
Follow the instructions in [README.md](https://github.com/MeteoSwiss/nwp-fdb-polytope-demo/blob/main/README.md#Installation-1) to install the necessary dependencies.

## Configuring Access to Polytope
To access ICON data via MeteoSwiss's Polytope, you need a Polytope offline token provided by MeteoSwiss. If you do not already have a token, you can request one [here](https://meteoswiss.atlassian.net/wiki/spaces/IW2/pages/327780397/Polytope#Offline-token-authentication). Then, create a new `config.yml` file based on [`config_example.yml`](config_example.yml), and replace <meteoswiss_key> with your access token there. 

In [ ]:
import os
import yaml

def load_config(path="config.yml"):
    if not os.path.exists(path):
        raise FileNotFoundError("Missing config.yml. Please create one based on config_example.yml")
    with open(path, "r") as f:
        return yaml.safe_load(f)

config = load_config()

#ICON-CSCS Polytope credentials
os.environ["POLYTOPE_USER_KEY"] = config["meteoswiss"]["key"]
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-depl.mchml.cscs.ch"

## Selecting date and time of the forecast

The FDB containing real-time data typically **holds only the most recent day of forecasts**. Therefore, it is necessary to specify the current date and select a corresponding forecast time in the past.

In [ ]:
from datetime import datetime, timedelta

# Current time
now = datetime.now()

# Subtract 12 hours
past_time = now - timedelta(hours=12)

# Round down to the nearest multiple of 6
rounded_hour = (past_time.hour // 6) * 6
rounded_time = past_time.replace(hour=rounded_hour, minute=0, second=0, microsecond=0)

# Format as YYYYMMDD and HHMM
date = rounded_time.strftime('%Y%m%d')
time = rounded_time.strftime('%H%M')
date,time

## Define the request

While other notebooks in this folder demonstrate how to use Polytope for feature extraction (e.g., extracting subsets or regions of interest), this example shows that we can also access the full horizontal grid directly via Polytope. To access the data containing the complete horizontal field, we need to define a MARS request **without** the `feature` key. The `meteodatalab.mars` module provides tools to help build valid MARS requests in the context of MeteoSwiss.  

This example fetches **2-m temperature** from **ICON-CH1-EPS** at the **surface**, for the **control forecast** (`type="cf"`), at the selected run date/time.

In [ ]:
from meteodatalab import mars

request = mars.Request(
    param="T_2M",
    date=date,
    time=time,
    model=mars.Model.ICON_CH1_EPS,
    levtype=mars.LevType.SURFACE,
    type="cf",
    step=0
)

## Data retrieval
Now we use [earthkit.data](https://earthkit-data.readthedocs.io/en/latest/) to load the data and convert it into an [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html). Since we request the full horizonal data, we need to make sure to access the collection `mch`.

In [ ]:
import earthkit.data as ekd
ds = ekd.from_source(
    "polytope",
    "mch",
    request.to_polytope(),
    stream=False
).to_xarray()

## Plotting

We use the library [earthkit.plots](https://earthkit-plots.readthedocs.io/en/latest/) to plot the data. Moreover, we can specify a styling with [earthkit.plots.styles](https://earthkit-plots.readthedocs.io/en/latest/examples/guide/04-styles.html#4.-Styles) to show a color spectrum from red to blue and convert the unit to Celsius for better readability.

In [ ]:
from earthkit.plots import Map
from earthkit.plots.styles import Style
from earthkit.plots.geo import bounds, domains
import cartopy.crs as ccrs

xmin, xmax = 5.5, 11   # Longitude bounds
ymin, ymax = 45.5, 48   # Latitude bounds

bbox = bounds.BoundingBox(xmin, xmax, ymin, ymax, ccrs.Geodetic())
domain = domains.Domain.from_bbox(
    bbox=bbox,
    name="CH2"
)

style = Style(colors="RdBu_r", units="celsius")
chart = Map(domain=domain)
chart.point_cloud(ds["T_2M"], x="longitude", y="latitude", style=style)

chart.coastlines()
chart.borders()
chart.gridlines()


chart.title(f"2-Meter Temperature at {rounded_time.isoformat()}")

chart.legend()

chart.show()